# Iterations and comprehensions
## I. 基本概念
### I.1 相关对象和工具
1.  <font color=blue>**iterable**</font>: 
   - 支持iter call的对象，也就是定义了<code>\_\_iter__</code> mehtod，且其返回值是一个iterator类型的对象。<code>iter(obj)</code>会invoke <code>obj.\_\_iter__() </code>
   - 包括：
     1. 属于sequence的那些类型，如：list, string, tuple
     2. file object
     3. <font color=orange>**dictionary, keys of dictionary, range**</font>, 以及<font color=orange>**map, zip, emumerate, filter的返回值**</font>，等
2. <font color=blue>**iterator**</font>: 
   - 按下面规则定义了<code>\_\_next__</code> method的对象都是iterator。
     - <font color=blue>**规则**：<code>\_\_next__</code>每次返回对象中的一个元素，直到遍历所有元素后raise <code>StopIteration exception</code></font>
   - built-in iterable在被iter()invoke之后得到的返回值就是典型的iterator
   - 自定义iterable时，必须定义对应的iterator

3. <font color=blue>**既是iterable又是iterator**</font>: 
   - 如果一个iterable对象的<code>\_\_iter__</code>method返回值是它自身，且又定义了<code>\_\_next__</code>method，且那么它**既是iterable又是iterator**。

#### <font color=deeppink>概念再区分：</font>
1. <font color=green>**既是iterable又是iterator：**</font>
   - 定义了<code>\_\_iter__</code> method的iterable对象。如果且它还同时定义了<code>\_\_next__</code> method，那么该iterable也是iterator。
2. <font color=green>**单iterable：**</font>
   - 定义了<code>\_\_iter__</code> method的iterable对象。如果<code>\_\_iter__</code> method的返回值是一个新的iterator对象，该对象中定义了<code>\_\_next__</code> method。
3. <font color=green>**单iterator：**</font>
   - 定义了<code>\_\_next__</code> method，但没有定义<code>\_\_iter__</code> method。

In [16]:
# 定义一般的iterable
class CountUpToIterable:
    def __init__(self, limit):
        self.limit = limit
    
    def __iter__(self):
        return CountUpToIterator(self.limit)  # Returns new iterator

# 定义对应的iterator
class CountUpToIterator:
    def __init__(self, limit):
        self.limit = limit
        self.current = 0
    
    def __next__(self):
        if self.current >= self.limit:
            raise StopIteration
        self.current += 1
        return self.current

# Can iterate multiple times
counter = CountUpToIterable(3)
print(list(counter))  # [1, 2, 3]
print(list(counter))  # [1, 2, 3] - works again!

[1, 2, 3]
[1, 2, 3]


- <font color=blue>**iteration tools**</font>: 
  - 是使用iterable对象的工具。他们根据**iteration protocol**中的规则来用iterable构造iterator，并执行遍历。
    - <font color=red>**iterable**是遍历的对象，而**iterator**是为了执行遍历而由iteration tool生成的临时对象。</font>
  - 包括：for loop, list comprehension, map, zip, range, etc
- <font color=green>**特点：通常情况下，在实现遍历的不同方式中，一些built-in iterable + iteration tool用C实现，运行效率高。**</font>
  - **fast**：比如list comprehension和map的实现就做了优化，runs at C language sppeed inside Python.
    - 同样的代码内容，list comprehension和map都比for loop快
    - list comprehension又比map快
  - <font color=norange>**memory efficient**</font>: iterator的迭代是lazy的，因为每次call<code>\_\_next__</code>才会提供下一个结果，不会一次生成所有element sequence来占用存储空间。

### I.2 iteration tool的工作方式
#### 1. iteration protocol
- 由两个主要的组件：<code>\_\_iter__</code> method和<code>\_\_next__</code> method
- protocol工作的规则是：
  1. iteration tool会先call <code>iter(iterable)</code>，得到其返回值iterator
     - <font color=orange>有的iterable自身就是iterator，此时<code>iter(iterable)</code>的返回值是<code>self</code>，就是对象自身。比如：file object</font>
     - <font color=orange>**list, dictionary**等常规的iterable都返回新的iterator对象。</font>
  2. 开始遍历后：iteration tool会反复call <code>iterator.\_\_next__()</code>，得到其返回的object。
     - 如果返回正常的object，则按照代码中的需要将其用于计算过程，并继续下一次遍历。
     - 如果返回了<code>StopIteration exception</code>则结束遍历。

#### 2. iteration tool用在iterable和iterator上的不同结果
- <font color=green>**非iterator的iterable支持反复遍历，iterator只能做一次遍历。**</font>
  - <font color=blue>**非iterator的iterable对象**</font>：
    - 每次iteration tool invoke <code>iter(iterable)</code>的时候，都会返回一个新的iterator对象，每个iterator都可以遍历一次。
  - <font color=blue>**iterable同时是iterator**</font>：
    - iteration tool invoke <code>iter(iterator)</code>的时候，返回该对象自己。每次call<code>\_\_next__</code>也是在对象自己的元素中遍历，直到遇到<code>StopIteration</code>之后遍历结束。
    - 再次invoke<code>iter(iterator)</code>不会重启遍历

In [2]:
# 既是iterable，又是iterator
class CountUpTo:
    def __init__(self, limit):
        self.limit = limit
        self.current = 0
    
    def __iter__(self):
        return self  # Returns self, making it an iterator
    
    def __next__(self):
        if self.current >= self.limit:
            raise StopIteration
        self.current += 1
        return self.current

# Can only iterate once
counter = CountUpTo(3)
print(list(counter))  # [1, 2, 3]
print(list(counter))  # [] - already exhausted!

# Need to create new instance for another iteration
counter2 = CountUpTo(3)
print(list(counter2))  # [1, 2, 3]

[1, 2, 3]
[]
[1, 2, 3]


### I.3 iteration tool的适用对象
- 所有支持iteration protocol的对象: iterable和generator
  1. <font color=green>**iterable**</font>
     - 单iterable或者同时是iterator的iterable都可以
     - 但是<font color=red>单iterator</font>不行，因为iteration protocol需要<code>\_\_iter__</code> method
  2. <font color=green>**generator**</font>
     - 在用yield作为返回值定义function时，python会自动帮generator加上<code>\_\_iter__</code> 和<code>\_\_next__</code>这两个method，设置<code>\_\_iter__</code>的返回值为对象本身，并根据函数体中的code内容<font color=orange>(从yield后一行到再次执行完yield)</font>来设置<code>\_\_next__</code>的返回值。

In [18]:
def count_up_to(n):
    count = 1
    while count <= n:
        yield count
        count += 1

counter = count_up_to(4)
print(counter.__iter__, counter.__next__)

<method-wrapper '__iter__' of generator object at 0x76b221255ee0> <method-wrapper '__next__' of generator object at 0x76b221255ee0>


## II. 典型的iterable

### II.1 string, list, tuple, dictionary和range等只是iterable
- iteration tool工作方式：
  - for loop会将iterable，这里的file object，传给<code>iter()</code>。
  - <code>iter(file_obj)</code>返回一个iterator。iterator自带了<code>next()</code> method.
  - for会在每次遍历中invoke <code>next()</code> method来执行遍历的内容。

### II.2 file object, generator以及<code>map(), zip()和filter()</code>等函数的返回值既是iterable，又是iterator
- <code>for line in file: ...</code>
- iteration tool工作方式：
  - file作为iterator自带了<code>next()</code> method.
  - for会在每次遍历中invoke <code>next()</code> method来执行遍历的内容。

In [3]:
# 用for loop按行读文件
with open('script.py', 'r') as f:
    print(iter(f) is f)
    for line in f:
        print(line.upper(), end='') # 文档每行结尾有'\n'，所以要suppress print中默认结尾加'\n'

True
# -*- CODING: UTF-8
IMPORT SYS
PRINT(SYS.PATH)

X = '文本'
Y = '\XC4\XE8'
FOR I IN X, Y:
    PRINT(F'{I}, {LEN(I)},{I.ENCODE()},  {I.ENCODE(''UTF-8)}')



## III. 典型的iteration tools
- 


### III.1 for loop
略

### III.2 各种形式的comprehension
- 常见形态：
  - <font color=blue>**list comprehension**</font>: <code>[expression(i) for i in iterable]</code>
  - <font color=blue>**dict comprehension**</font>: <code>{key: v for key, v in zip(x, y)}</code>
  - <font color=blue>**generator comprehension**</font>: <code>(expression(i) for i in iterable)</code>
    - <font color=orange>注：用'()'得到的是generator comprehension，不是tuple comprehension</font>。
  - <font color=blue>**tuple comprehension**</font>: <code>tuple(expression(i) for i in iterable)</code>
    - <font color=green>用tuple comprehension的场景：生成的tuple sequence是immutable</font>
#### III.2.1 list comprehension
- 用法
  - 形态：<code>[func(i) for i in iterable]</code>
  - 以file为例：
    - <code>[func(line) for line in open('file_name')]</code>
    - 特点是不会一开始就读出来整个文档，而是处理一行读取一行。
    - 下面这种方式就是要先读出来整个文档，然后再逐行处理：
    ```python
    lines = open('file_name').readlines()
    [func(line) for line in lines]         # 虽然都是list comprehension，但执行效果不同
    ```

- 扩展形态
  - <font color=green>**嵌套方式依据python从左到右执行statement的原则来判断。**</font>
  1. 加条件
     - 形式：<code>[expression(i) for i in iterable if func(i)]</code>
  2. 双层遍历：
     - 形式1：<code>[expression(j)  for i in iterable for j in i]</code>
     - 形式2：<code>[expression(i,j)  for i in iterable_1 for j in iterable_2]</code>

#### III.2.2  generator comprehension
- **generator comprehension和其他几种常见comprehension的最主要区别**：
  - generator comprehension的返回值是一个generator，它在遍历的时候是lazy的
  - list, dict, tuple comprehension的返回值是对应的sequence object。

In [4]:
my_list = [1, 3, 5, 9, 2, 6]
filtered_list = [item for item in my_list if item > 3]
filtered_generator = (item for item in my_list if item > 3)
print(filtered_list)
print(filtered_generator)

[5, 9, 6]
<generator object <genexpr> at 0x76b22c3802e0>
